In [1]:
# Kaggle API for downloading the datasets
!pip install --upgrade -q kaggle

# Optuna for parameter search
#!pip install -q optuna

!pip install --upgrade xgboost

# upgrask sklearn
!pip install --upgrade scikit-learn

!pip install category_encoders

'''
!pip install catboost
# lighgbm gpu compatible
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
'''

     |████████████████████████████████| 166.7 MB 15 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
     |████████████████████████████████| 22.3 MB 1.7 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 80 kB 5.4 MB/s 


'\n!pip install catboost\n# lighgbm gpu compatible\n!git clone --recursive https://github.com/Microsoft/LightGBM\n! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;\n'

In [2]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
import pickle
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from scipy.optimize import minimize

#import lightgbm as lgb
import xgboost as xgb
#import catboost as ctb

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [3]:
'''
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
'''

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# get a list of kaggle competitions
#!kaggle competitions list

# download the target competition
!kaggle competitions download 'tabular-playground-series-aug-2021'

# unzip the files if needed
## !unzip tabular-playground-series-jan-2021.zip


  0% 0.00/325k [00:00<?, ?B/s]
100% 325k/325k [00:00<00:00, 45.0MB/s]
 98% 55.0M/55.9M [00:01<00:00, 26.7MB/s]
100% 55.9M/55.9M [00:01<00:00, 32.9MB/s]
 95% 89.0M/93.3M [00:04<00:00, 22.3MB/s]
100% 93.3M/93.3M [00:04<00:00, 22.1MB/s]


In [4]:
#! kaggle datasets download lilkaskitc/30-days-pseudo
#! unzip 30-days-pseudo.zip

# Parameters

In [21]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 9000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [22]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [7]:
# load the data
train = pd.read_csv("train.csv.zip")
test = pd.read_csv("test.csv.zip")
submission = pd.read_csv("sample_submission.csv.zip")

# Preprocessing

In [8]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# XGB

In [23]:
xgb_params = {
    'objective': 'reg:squarederror',    
    'seed': SEED,   
    'n_estimators': N_ESTIMATORS,
    'importance_type': 'total_gain',
    'tree_method': 'gpu_hist',    
    
    'learning_rate': 0.0018331925260194747,
     'subsample': 0.5999280340279187,
      'colsample_bytree': 0.8309679692775827, 
      'max_depth': 11,
       'lambda': 9.484816540848616,
       'alpha': 5.787546831727187, 
       'min_child_weight': 203.58662168918866
}

In [25]:

xgb_pred = np.zeros(test.shape[0])

X_train, y_train = train[scale_features], train[target]
#X_train = apply_noise_row(X_train)

X_test = test[scale_features]



# XGB       

xgb_params['seed'] = SEED

pre_model = xgb.XGBRegressor(**xgb_params)
pre_model.fit(
    X_train, 
    y_train,
    
    verbose=VERBOSE
      )


xgb_params2 = xgb_params.copy()
xgb_params2['lambda'] *= 0.9
xgb_params2['alpha'] *= 0.9
xgb_params2['learning_rate'] *= 0.1
model = xgb.XGBRegressor(**xgb_params2)
model.fit(
        X_train, y_train,
      
        verbose=VERBOSE,
        xgb_model=pre_model
        )    



xgb_pred += model.predict(X_test) 


np.save("xgb_pred.npy", xgb_pred)

# Submission

In [26]:
submission[target] = xgb_pred
submission.to_csv("submission.csv", index=False)

submission

,id,loss
0,250000,7.613151
1,250001,5.161349
2,250002,8.685816
3,250003,7.457319
4,250004,6.975274
...,...,...
149995,399995,7.807847
149996,399996,7.720120
149997,399997,5.963680
149998,399998,5.428344


In [28]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>